In [2]:
!pip install -U ultralytics
import os, json, shutil, math
from pathlib import Path


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
TRAIN_IMAGES = "/content/drive/MyDrive/Dental-X/train"
TRAIN_JSON   = "/content/drive/MyDrive/Dental-X/train/_annotations.coco.json"

VAL_IMAGES   = "/content/drive/MyDrive/Dental-X/valid"
VAL_JSON     = "/content/drive/MyDrive/Dental-X/valid/_annotations.coco.json"


TASK = "segment"
PROJECT_OUT = "/content/drive/MyDrive/Dental-X/yolo"
os.makedirs(PROJECT_OUT, exist_ok=True)


In [5]:
from collections import defaultdict
from pathlib import Path
import json, os

def _xywh_to_yolo(x, y, w, h, W, H):

    cx = (x + w/2.0) / W
    cy = (y + h/2.0) / H
    return cx, cy, w / W, h / H

def _seg_to_yolo_polys(seg, W, H):
    yolo_polys = []
    for poly in seg:

        if len(poly) < 6 or len(poly) % 2 != 0:
            continue
        pts = []
        for i in range(0, len(poly), 2):
            x = poly[i]   / W
            y = poly[i+1] / H
            pts += [x, y]
        if len(pts) >= 6:
            yolo_polys.append(pts)
    return yolo_polys

def coco_to_yolo_split(img_dir, ann_json, out_labels_dir, task="detect"):
    img_dir = Path(img_dir)
    out_labels_dir = Path(out_labels_dir)
    out_labels_dir.mkdir(parents=True, exist_ok=True)

    with open(ann_json, "r") as f:
        coco = json.load(f)


    cats = sorted(coco["categories"], key=lambda c: c["id"])
    catid_to_idx = {c["id"]: i for i, c in enumerate(cats)}
    names = [c["name"] for c in cats]

    anns_per_img = defaultdict(list)
    for ann in coco["annotations"]:
        anns_per_img[ann["image_id"]].append(ann)

    images_by_id = {im["id"]: im for im in coco["images"]}

    missing_images = 0
    for img_id, im in images_by_id.items():
        file_name = im["file_name"]
        W, H = im["width"], im["height"]
        img_path = img_dir / file_name
        if not img_path.exists():
            missing_images += 1
            continue

        label_path = out_labels_dir / (Path(file_name).stem + ".txt")
        lines = []

        for ann in anns_per_img.get(img_id, []):
            cid = catid_to_idx.get(ann["category_id"])
            if cid is None:
                continue

            if task == "detect":
                if "bbox" not in ann:
                    continue
                x, y, w, h = ann["bbox"]
                cx, cy, nw, nh = _xywh_to_yolo(x, y, w, h, W, H)
                lines.append(f"{cid} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")

            elif task == "segment":

                seg = ann.get("segmentation", [])
                if isinstance(seg, list) and len(seg) > 0:
                    polys = _seg_to_yolo_polys(seg, W, H)
                    for pts in polys:

                        lines.append(" ".join([str(cid)] + [f"{v:.6f}" for v in pts]))
                elif "bbox" in ann:
                    x, y, w, h = ann["bbox"]
                    cx, cy, nw, nh = _xywh_to_yolo(x, y, w, h, W, H)
                    lines.append(f"{cid} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")

        with open(label_path, "w") as f:
            f.write("\n".join(lines))

    return names, missing_images


labels_base = Path(PROJECT_OUT) / "labels"
names_train, miss_tr = coco_to_yolo_split(TRAIN_IMAGES, TRAIN_JSON, labels_base / "train", TASK)
names_val,   miss_va = coco_to_yolo_split(VAL_IMAGES,   VAL_JSON,   labels_base / "val",   TASK)


names = names_train if names_train else names_val
print("Classes:", names)
print("Missing train images:", miss_tr, "Missing val images:", miss_va)


Classes: ['Tooth', 'Caries', 'Crown', 'Filling', 'Implant', 'Mandibular Canal', 'Missing teeth', 'Periapical lesion', 'Retained root', 'Root Canal Treatment', 'Root Piece', 'impacted tooth', 'maxillary sinus']
Missing train images: 0 Missing val images: 0


In [6]:
from pathlib import Path
import yaml, shutil

images_base = Path(PROJECT_OUT) / "images"
labels_base = Path(PROJECT_OUT) / "labels"
(images_base / "train").mkdir(parents=True, exist_ok=True)
(images_base / "val").mkdir(parents=True, exist_ok=True)


def _link_or_copy(src_dir, dst_dir):
    src_dir, dst_dir = Path(src_dir), Path(dst_dir)
    for p in src_dir.iterdir():
        if not p.is_file():
            continue
        dst = dst_dir / p.name
        try:
            os.symlink(p, dst)
        except Exception:
            shutil.copy2(p, dst)

_link_or_copy(TRAIN_IMAGES, images_base / "train")
_link_or_copy(VAL_IMAGES,   images_base / "val")

data = {
    "path": str(Path(PROJECT_OUT).resolve()),
    "train": "images/train",
    "val": "images/val",
    "names": names,
}
if TASK == "segment":
    data["task"] = "segment"

yaml_path = Path(PROJECT_OUT) / "data.yaml"
with open(yaml_path, "w") as f:
    yaml.safe_dump(data, f, sort_keys=False)

print("Wrote", yaml_path)


Wrote /content/drive/MyDrive/Dental-X/yolo/data.yaml


In [7]:
from ultralytics import YOLO

model_name = "yolov8s-seg.pt" if TASK == "segment" else "yolov8s.pt"

model = YOLO(model_name)
results = model.train(
    data=str(yaml_path),
    epochs=100,
    imgsz=640,
    batch=16,
    project=PROJECT_OUT,
    name="yolo_runs",
    patience=50,
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.186 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Dental-X/yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, ker

In [9]:
from ultralytics import YOLO


model.val(data=str(yaml_path), imgsz=640)


TEST_DIR = "/content/drive/MyDrive/Dental-X/all_images"
pred = model.predict(source=TEST_DIR, imgsz=640, save=True, project=PROJECT_OUT, name="preds", stream=True)
#ONNX
model.export(format="onnx", opset=12)

Ultralytics 8.3.186 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 24.3±5.5 MB/s, size: 35.7 KB)
val: Scanning /content/drive/MyDrive/Dental-X/yolo/labels/val.cache... 260 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 260/260 400484.4it/s 0.0s
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per i

'/content/drive/MyDrive/Dental-X/yolo/yolo_runs/weights/best.onnx'